# Import

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import json

# Scraping Name and Score

Scraping name and score from the website https://pvpoke.com/rankings/. <br>
Store Data in raw_data/pokemon_name_rank.json. 

In [2]:
def scrape_rankings_names_scores_to_json():
    # Set up the Selenium WebDriver
    driver = webdriver.Chrome()

    try:
        # Navigate to the rankings page
        url = 'https://pvpoke.com/rankings/'
        driver.get(url)

        # Wait for the page to load completely
        time.sleep(15)

        # Get the page source and parse it with BeautifulSoup
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Extract Pokémon rankings data
        pokemon_blocks = soup.find_all('div', class_='rank')
        data = []

        for pokemon in pokemon_blocks:
            name = pokemon.find('span', class_='name').text if pokemon.find('span', class_='name') else "No name"
            score = pokemon.find('div', class_='rating score-rating').text if pokemon.find('div', class_='rating score-rating') else "No score"

            data.append({"name": name, "score": score})

        # Filter out entries with 'No score'
        data = [entry for entry in data if entry['score'] != "No score"]

        # Save data to a JSON file in the raw_data folder
        output_path = 'raw_data/pokemon_name_rank.json'
        with open(output_path, 'w') as f:
            json.dump(data, f, indent=4)

        print(f"Data has been saved to {output_path}")

    finally:
        # Quit the driver
        driver.quit()

In [3]:
# Call the function
scrape_rankings_names_scores_to_json()

Data has been saved to raw_data/pokemon_name_rank.json


In [6]:
def get_pokemon_stats_and_moves():
    # Load the names from the JSON file
    with open('raw_data/pokemon_name_rank.json', 'r') as f:
        pokemon_data = json.load(f)

    results = []
    for entry in pokemon_data:
        name = entry['name'].replace(' ', '_').replace('.', '').replace("'", '').lower()
        url = f'https://pvpoke.com/rankings/all/1500/overall/{name}/'

        driver = webdriver.Chrome()
        try:
            driver.get(url)
            time.sleep(8)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            # Get stats
            stats_block = soup.find('div', class_='stat-block')
            stats = {}
            if stats_block:
                for stat in stats_block.find_all('div', class_='stat'):
                    stat_name = stat.find('span', class_='label').text.strip()
                    stat_value = stat.find('span', class_='value').text.strip()
                    stats[stat_name] = stat_value

            # Get moves
            fast_moves = soup.find('div', id='fast-moves')
            charge_moves = soup.find('div', id='charge-moves')
            num_fast_moves = len(fast_moves.find_all('div', class_='move')) if fast_moves else 0
            num_charge_moves = len(charge_moves.find_all('div', class_='move')) if charge_moves else 0

            results.append({
                'name': entry['name'],
                'score': entry['score'],
                'url': url,
                'stats': {
                    'Attack': stats.get('Attack', None),
                    'Defense': stats.get('Defense', None),
                    'HP': stats.get('HP', None),
                    'Stat Product': stats.get('Stat Product', None)
                },
                'moves': {
                    'num_fast_moves': num_fast_moves,
                    'num_charge_moves': num_charge_moves
                }
            })
            print(results[-1])
        finally:
            driver.quit()

    # Save results
    with open('raw_data/pokemon_stats_and_moves.json', 'w') as f:
        json.dump(results, f, indent=4)

    print("Stats and moves have been saved to raw_data/pokemon_stats_and_moves.json")

# Call the function
get_pokemon_stats_and_moves()

{'name': 'Clodsire', 'score': '94.4', 'url': 'https://pvpoke.com/rankings/all/1500/overall/clodsire/', 'stats': {'Attack': None, 'Defense': None, 'HP': None, 'Stat Product': None}, 'moves': {'num_fast_moves': 0, 'num_charge_moves': 0}}
{'name': 'Diggersby', 'score': '93.6', 'url': 'https://pvpoke.com/rankings/all/1500/overall/diggersby/', 'stats': {'Attack': None, 'Defense': None, 'HP': None, 'Stat Product': None}, 'moves': {'num_fast_moves': 0, 'num_charge_moves': 0}}
{'name': 'Diggersby', 'score': '93.6', 'url': 'https://pvpoke.com/rankings/all/1500/overall/diggersby/', 'stats': {'Attack': None, 'Defense': None, 'HP': None, 'Stat Product': None}, 'moves': {'num_fast_moves': 0, 'num_charge_moves': 0}}
{'name': 'Forretress', 'score': '93.6', 'url': 'https://pvpoke.com/rankings/all/1500/overall/forretress/', 'stats': {'Attack': None, 'Defense': None, 'HP': None, 'Stat Product': None}, 'moves': {'num_fast_moves': 0, 'num_charge_moves': 0}}
{'name': 'Forretress', 'score': '93.6', 'url': '

KeyboardInterrupt: 